In [1]:
import pandas as pd
import email


In [2]:
#datasets
all_data = "emails.csv"

In [3]:
import yfinance as yf
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("YFinanceToSpark").getOrCreate()

# Fetch data from yfinance
data = yf.download("AAPL", start="2020-01-01", end="2022-03-01", interval="1d")

# Convert the pandas DataFrame to a PySpark DataFrame
df = spark.createDataFrame(data.reset_index())

# Show the PySpark DataFrame
df.show()

# Stop the SparkSession
spark.stop()


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark=SparkSession.builder.appName("Gold").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 21:01:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark

In [22]:
## Lets define the dataset column type 
df_pyspark = spark.read.option('header', 'true').csv(all_data, inferSchema=True)
df_pyspark.show(5)

+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|
|           Date: Mon| 14 May 2001 16:3...|
|From: phillip.all...|                null|
|To: tim.belden@en...|                null|
|           Subject: |                null|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Calculate the size of the DataFrame in bytes
size_bytes = df_pyspark.rdd.map(lambda x: x.__sizeof__()).reduce(lambda a, b: a + b)

# Convert the size to gigabytes
size_gb = size_bytes / (1024.0 ** 3)

# Print the size in gigabytes
print("DataFrame size: {:.2f} GB".format(size_gb))

In [21]:
df_pyspark = spark.createDataFrame(x, ["file", "message"])
df_pyspark.show(5)

ParseException: 
Syntax error at or near end of input(line 1, pos 7)

== SQL ==
message
-------^^^


In [11]:
df_pyspark.show(10)

+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|
|allen-p/_sent_mai...|Message-ID: <1546...|
|allen-p/_sent_mai...|Message-ID: <2421...|
|allen-p/_sent_mai...|Message-ID: <1350...|
|allen-p/_sent_mai...|Message-ID: <3092...|
+--------------------+--------------------+



In [20]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import udf

# Define a UDF to parse email message and extract sender and receiver
def parse_email_message(message):
    e = email.message_from_string(message)
    sender = e.get('From')
    reciever = e.get('To')
    return (sender, reciever)

# Define the schema for the struct type
email_schema = StructType([
    StructField('Sender', StringType(), True),
    StructField('Receiver', StringType(), True)
])

# Convert the UDF to return a struct type
parse_email_message_udf = udf(parse_email_message, email_schema)

# Add columns for sender and receiver using the UDF
df_pyspark = df_pyspark.withColumn('EmailData', parse_email_message_udf('message'))
df_pyspark = df_pyspark.withColumn('Sender', df_pyspark['EmailData']['Sender'])
df_pyspark = df_pyspark.withColumn('Receiver', df_pyspark['EmailData']['Receiver'])
# Select only the necessary columns

df_pyspark.show(15)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                file|             message|           EmailData|              Sender|            Receiver|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|{phillip.allen@en...|phillip.allen@enr...|tim.belden@enron.com|
|allen-p/_sent_mai...|Message-ID: <1546...|{phillip.allen@en...|phillip.allen@enr...|john.lavorato@enr...|
|allen-p/_sent_mai...|Message-ID: <2421...|{phillip.allen@en...|phillip.allen@enr...|leah.arsdall@enro...|
|allen-p/_sent_mai...|Message-ID: <1350...|{phillip.allen@en...|phillip.allen@enr...|randall.gay@enron...|
|allen-p/_sent_mai...|Message-ID: <3092...|{phillip.allen@en...|phillip.allen@enr...|greg.piper@enron.com|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [14]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import udf

# Define a UDF to parse email message and extract sender and receiver
def parse_email_message(message):
    e = email.message_from_string(message)
    e = email.message_from_string(message)
    sender = e.get('From')
    reciever = e.get('To')
    #print(f"Message: {message}")
    #print(f"Sender: {sender}")
    #print(f"Receiver: {reciever}")
    #return (sender, reciever)

# Define the schema for the struct type
email_schema = StructType([
    StructField('Sender', StringType(), True),
    StructField('Receiver', StringType(), True)
])

# Convert the UDF to return a struct type
parse_email_message_udf = udf(parse_email_message, email_schema)

# Add columns for sender and receiver using the UDF
df_pyspark = df_pyspark.withColumn('EmailData', parse_email_message_udf('message'))
df_pyspark = df_pyspark.withColumn('Sender', df_pyspark['EmailData']['Sender'])
df_pyspark = df_pyspark.withColumn('Receiver', df_pyspark['EmailData']['Receiver'])

# Show the first few rows of the DataFrame
df_pyspark.show(5)


+--------------------+--------------------+---------+------+--------+
|                file|             message|EmailData|Sender|Receiver|
+--------------------+--------------------+---------+------+--------+
|allen-p/_sent_mai...|Message-ID: <1878...|     null|  null|    null|
|allen-p/_sent_mai...|Message-ID: <1546...|     null|  null|    null|
|allen-p/_sent_mai...|Message-ID: <2421...|     null|  null|    null|
|allen-p/_sent_mai...|Message-ID: <1350...|     null|  null|    null|
|allen-p/_sent_mai...|Message-ID: <3092...|     null|  null|    null|
+--------------------+--------------------+---------+------+--------+

